In [1]:
import os  #운영체제와의 상호작용을 돕는 다양한 기능을 제공하는 모듈  
import glob # 파일들의 리스트를 뽑을 때 사용 (파일의 경로명을 이용)
import copy # 리스트 래퍼런스로 인한 문제를 해결하기 위해 사용가능
import random # 파이썬에서 난수(random number)를 구할 수 있는 모듈
import math # 수학과 관련된 다양한 함수들과 상수들이 미리 정의 되어 있음
import cv2 # openCV 사용할때는 cv2로 import함
import numpy as np # Matrix와 vector와 같은 Array 연산을 할 때 사용하는 라이브러리
from tqdm import tqdm # 즉석에서 progress bar를 생성해주고 함수나 반복문의 TTC (Time To Completion) 를 예측하는 파이썬 패키지
from skimage import io #이미지 처리에 특화된 python이미지 라이브러리이며 Numpy배열로 이미지 객체를 다룸
import matplotlib.pyplot as plt #다양한 데이터를 많은 방법으로 도식화 할 수 있도록 하는 파이썬 라이브러리
from PIL import Image #이미지 처리와 그래픽기능 제공하는 라이브러리 (픽셀 단위의 조작,마스킹,투명도 제어,이미지 필터)
from pathlib import Path # 파일시스템 경로를 단순한 문자열이 아닌 객체로 다루게 해주는 모듈로 파일의 존재성 여부 판단을 할 수 있음 
import sys # 파이썬 인터프리터가 제공하는 변수와 함수를 직접 제어할 수 있게 해주는 모듈

import openslide #digital pathology에서 whole slide image를 읽기 위한 간단한 인터페이스를 제공하는 라이브러리
import xml.etree.ElementTree as et #XML 데이터를 구문 분석하고 만들기 위한 단순하고 효율적인 API(프로그램들이 서로 상호작용하는 것을 도와주는 매게체)를 구현하는 모듈.


In [10]:
# xml파일을 마스크로 만드는 함수
def xml2mask(xml_fn, shape):    # xml_fn : ln 혹은 noln에 대한 annotation 좌표들이 있는 xml파일의 경로?, shape : 슬라이드이미지의 shape
     board = np.zeros(shape[:2],dtype = np.uint8) # shape에서 행렬까지에 대한 정보만을 가지고 unit8 type으로 넘파이 어레이만듬(0)
     e = et.parse(xml_fn).getroot() # xml_fn파일을 parse(분석)하고 최상단 루트태그를 가져온다
     e = e.findall('Annotations') # root(e) 하위에 "Annotations"와 일치하는 모든 태그를 리스트로 리턴한다.
     assert (len(e) == 1), len(e) # 만약 리스트의 길이가 1이 아니면 에러 출력
     e = e[0] #e의 첫 번째 원소만을 e라는 변수에 저장
     contour_list = []   #윤곽에 대한 위치정보들을 저장할 리스트로 생각됨
     for ann in e:   # e = e.findall에서는 e가 [[x,y,z,s,d,f,,,]] 이러한 형태 였다고 생각되어짐 #ann 하나하나의 cluster, xml파일 열어보기
            ann = ann.findall('Coordinates')[0] # e의 원소를 하나씩 가져와서 ann에 해당하는 것의 하위에 Coordinate와 일치하는 태그들 중 첫번째 원소를 ann에 저장
            coord_list = [] #coordinate에 대한 것들을 리스트에 담기 위해 빈 리스트 생성
            for coord in ann: # ann에 안에 원소들을 하나씩 coord로 가져옴
                y,x = int(coord.get('Y').split('.')[0]), int(coord.get('X').split('.')[0])# Y혹은 X에 대응되는 value중 첫번 째원소를 .을 기준으로 나누어져 리스트로 반환 됨
                if div is not None: #div 태그 (영역을 나누거나 내용을 담는 container로서의 역할)가 있다면
                    y //= div   # y = y // div  #y를 div 로 나눠준 것에 정수만을 다시 y에 저장
                    x //= div   # xml은 level0을 기준으로 좌표를 갖고 있음 (level1로 마스크 얻고 싶으면 div = 2**1 )
                if y >= shape[0]:  # 만약 y가 shape[0], 즉 행의 개수보다 크거나 같으면
                    y = shape[0]-1  # 행의 개수 -1을 y값에 저장
                elif y < 0:     # 만약 y가 0보다 작으면 y에 0값 대입     # y축 범위를 유한하게 만들어주기 위한 과정인 것으로 생각됨
                    y = 0       
                if x >= shape[1]:  # y와 같은 과정을 x축에 대해서도 해줌
                    x = shape[1]-1
                elif x < 0:
                    x = 0
                    
                coord_list.append((x,y))     # coordinate에 대한 빈 list에 x,y정보에 대한 리스트를 append (x,y가 한쌍으로 튜플로 묶인)
            contour_list.append(coord_list)  # 윤곽에 대한 위치정보들을 담을 리스트에 좌표 정보들이 담긴 리스트를 append

     for coord_list in contour_list:  #annotation에 대한 위치정보들을 하나씩 coord_list로 가져와서
         #cv2.drawContours(이미지, [윤곽선], 윤곽선 인덱스, (B, G, R), 두께, 선형 타입)
         board = cv2.drawContours(board,[np.array(coord_list,dtype = np.int32)], -1, [255,255,255], -1) # 모든 원소가 0인 nd array에 그림
         #옵션(-1) 때문에 결국 mask얻는 역할함
     mask = board #윤곽선이 그려진 numpy array를 mask에 저장
     mask[mask == 255] = 1 #원소 값이 255인 것은 모두 1로 바꾸고
     mask[mask==0] = 0     #원소 값이 0인 것은 0으로 바꾼다. 
     return mask



In [23]:
board = np.zeros([5,3,4],dtype = np.uint8)
board[board == 0] = 23
print(board)
board.shape





[[[23 23 23 23]
  [23 23 23 23]
  [23 23 23 23]]

 [[23 23 23 23]
  [23 23 23 23]
  [23 23 23 23]]

 [[23 23 23 23]
  [23 23 23 23]
  [23 23 23 23]]

 [[23 23 23 23]
  [23 23 23 23]
  [23 23 23 23]]

 [[23 23 23 23]
  [23 23 23 23]
  [23 23 23 23]]]


(5, 3, 4)

In [30]:
seed = 42
np.random.seed(seed)
random.seed(seed) #random.random()은 [0,1) 사이의 실수로 난수를 구한다
#np.random.rand(4)

array([0.37454012, 0.95071431, 0.73199394, 0.59865848])

In [ ]:
#/mnt/hsyoo/paip2021/data/patch/train2/Col_0001/0/class0
root_dir = "Z:\\hsyoo\\"  #       #"/mnt/pathology/data/train"
modes = ["NOLN_metastasis", "LN_metastasis"] 
PATCH_SAVE = Path("/mnt/pathology/data/patch/train") # patch들을 저장할 경로
PSIZE = (224,224) # 하나의 patch size
levels = [1,2,3]  #
max_patches = 2000 # 생성할 패치의 수
 
ln_mrxs_paths = []  # LN슬라이드 이미지의 경로 (mrxs)
noln_mrxs_paths = [] # NOLN슬라이드 이미지의 경로 (mrxs)
ln_xml_paths = []   # LN 슬라이드에 대한 annotation 정보가 담긴 xml 파일의 경로
noln_xml_paths = [] # NOLN 슬라이드에 대한 annotation 정보가 담긴 xml 파일의 경로
for mode in modes:  #LN인지 NOLN인지 구분
    print(mode)
    if mode == "NOLN_metastasis":
        ln = False
    else:
        ln = True
    for patient in sorted(os.listdir(os.path.join(root_dir, mode))): #root_dir+mode에 있는 모든 파일과 폴더명 리스트를 반환
        if patient.split('.')[-1] == 'mrxs': # 하나씩 불러 뽑았을 때 그것이 mrxs(슬라이드이미지)라면 
            if ln == False:    #그 슬라이드 이미지가 NOLN이라면
                noln_mrxs_paths.append(os.path.join(root_dir, mode, patient)) #NOLN슬라이드 이미지들의 경로를 모아 놓을 리스트에 경로 추가
            else:              #그 슬라이드 이미지가 LN이라면
                ln_mrxs_paths.append(os.path.join(root_dir, mode, patient)) #LN슬라이드 이미지들의 경로를 모아 놓을 리스트에 경로 추가
        elif patient.split('.')[-1] == 'xml':        # 하나씩 불러와서 뽑았을 때 그것이 xml(annotation에 대한 파일)이라면
            if ln == False:                          #  그 xml이 LN에 대한 것이면
                noln_xml_paths.append(os.path.join(root_dir, mode, patient)) #LN xml파일들의 경로를 모아 놓을 리스트에 경로 추가
            else:                                     #  그 xml이 NOLN에 대한 것이면
                ln_xml_paths.append(os.path.join(root_dir, mode, patient))  #NOLN xml파일들의 경로를 모아 놓을 리스트에 경로 추가
#                
ln_xml_paths = ln_xml_paths[:29] + ln_xml_paths[30:] # 1번부터 30번째 xml파일 경로들과 30번째 이후의 xml파일들에 경로를 합침
print(len(noln_mrxs_paths), len(ln_mrxs_paths))
print(len(noln_xml_paths), len(ln_xml_paths))
print(len(ln_xml_paths))

In [33]:
t = 'sdasd.mrxs'

a = t.split('.')[-1]

print(a)

mrxs


In [ ]:
# level에 따른 슬라이드이미지의 shape을 load하는 함수?? (level마다 픽셀좌표가 달라지므로 그에 따른 shape정보도 달라지니까??)
def load_mrxs_shape(slide, level):   #슬라이드와 level에 대한 정보를 인수로 받음
    return [slide.level_dimensions[level][1], slide.level_dimensions[level][0]] 

#각 deep zoom level마다의 픽셀좌표(x,y) 리스트
# [1] : y 좌표 [0] : x 좌표

#level_dimensions
# A list of (pixels_x, pixels_y) tuples for each Deep Zoom level. level_dimensions[k] are the dimensions of level k.

In [ ]:
# annotation의 바깥 영역(no cancer)에 대한 mask를 얻는 함수??
def get_normal(mrxs_fn, xml_fn, level, shape): # mrxs_fn과 xml)fn, level정보, shape정보를 인수로 받음
    # 이미지 래핑(wrapping)
    # openslide.Openslide(filename)
    # Return an OpenSlide for whole-slide images and an ImageSlide for other types of images.
    # wholeslide이미지 에 대한 오픈슬라이드 이미지와 이미지의 다른 타입에 대한 이미지 슬라이드를 반환
    slide = openslide.OpenSlide(mrxs_fn)
    
    # read_region(location, level, size):
    # Return an RGBA Image containing the contents of the specified region
    # slide의 특정 region에 대해 넘파이 어레이로만듬 
    slide = np.array(slide.read_region((0,0), level, slide.level_dimensions[level]))[:,:,:3]
    #
    mask = slide[:,:,0].copy() #넘파이 어레이로 만든 slide의 R채널에 해당되는 것만 mask에 저장?
    
    mask[mask == 255] = 0  # mask에서 원소가 255인 것은 0으로 #흰배경 0으로
    mask[mask != 0] = 1    # mask에서 원소가 0이 아닌 것은 1로 #반전시켜서 binary이미지로 만듬?
                           # 흰배경이 아닌 부분 모두 1로 바꿈
    mask = mask - xml2mask(xml_fn,shape) #배경 제외한 전체 lymphnode - cancer인 부분의 윤곽만 그려져있는 이미지
    mask[mask == 255] = 1 # 255인 원소는 1로 바꿈
    return mask #흰배경인 부분은 모두 검정으로 바뀌고 lymphnode부분은 모두 흰색인데 cancer영역은 부분만
                 #검은색으로 그려져있을 것이라고 생각됨

In [ ]:
# 앞서 정의한 함수들을 사용해 patch를 생성하는 함수 
# (슬라이드 이미지, 마스크, LN NOLN여부, 저장할 파일이름, label정보)를 인수로 받음
def make_patch(slide, masks, mode, name, level, label):
    # masks의 원소 값중 1인 것들의 첫번째 차원의 index를 반환하고 그것에 개수에 해당하는 크기의 1차원넘파이 어레이를 만듬
    # np.arrange(5) => [0,1,2,3,4]
    idx = np.arange(len(np.where(masks == 1)[0])) 
    np.random.shuffle(idx) #크기 순으로 정렬되어 있는 원소의 index를 섞는다
    coord_y = np.where(masks == 1)[0][idx] #mask(cancer)부분 중 랜덤으로 뽑힌 y좌표
    coord_x = np.where(masks == 1)[1][idx] # mask(cancer)부분 중 랜덤으로 뽑힌 x좌표
    if len(idx) > max_patches:         # 뽑으려는 patch수보다 원소값이 1인 것들이 더 많다면
        for i in range(max_patches):   # max_patch개수 만큼 반복문 실행하여 패치 생성 후 저장
            #******slide.read_region******* mrxs랑 xml파일 열어서 계속 연습******************************************
            x = coord_x[i]*(2**(level)) - (PSIZE[0]//2)*(2**(level)) # 뽑으려는 패치의 x축에 대한 중심좌표
            y = coord_y[i]*(2**(level)) - (PSIZE[0]//2)*(2**(level)) #  뽑으려는 패치의 y축에 대한 중심좌표
            img = np.array(slide.read_region((x,y), level, PSIZE))[:,:,:3] # 전체슬라이드에서 패치에 해당하는 부분만 RGB채널로 넘파이 어레이 형태로 만들어 이미지변수에 저장
            img = img.astype(np.uint8) #이미지의 type을 양수만 가질 수 있는 uint8로 변경
            #색상 공간 변환 함수(cv2.cvtcolor)로 이미지의 색상 공간을 변경
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) #Blue Green Red채널로
            # np.unique는 입력된 배열에서 중복되지 않는 고유한 요소들의 배열을 리턴한다
            if (0 in np.unique(img[:,:,0])) and (0 in np.unique(img[:,:,1])) and (0 in np.unique(img[:,:,2])):
                continue # (background)원소의 0이 있으면 반복문 다음 순서 실행??
            
            num = format(i, '05') # i값을 5자리 숫자로 맞추고 i값이 5자리보다 작다면 앞에 0붙여서 5자리로 만듬
            ipath = Path(os.path.join(PATCH_SAVE, mode, name, str(level), f"{label}/img", f"{name}_{num}.png"))
            ipath.parent.mkdir(parents=True, exist_ok=True) #파일 생성할 경로(ipath)에 폴더가 없다면 폴더 생성??
            
            try:
                cv2.imwrite(str(ipath), img) #ipath경로에 img를 저장한다 (cv2.imwrite : 변환된 이미지나 동영상의 특정 프레임 저장)
            except:
                continue
    else:   # 뽑으려는 patch수보다 원소값이 1인 것들이 더 적다면
        for i in range(len(idx)): # 원소값이 1인 것들의 수(idx의크기) 만큼 반복문 실행하여 패치 생성 및 저장
            x = coord_x[i]*(2**(level)) - (PSIZE[0]//2)*(2**(level))
            y = coord_y[i]*(2**(level)) - (PSIZE[0]//2)*(2**(level))
            img = np.array(slide.read_region((x,y), level, PSIZE))[:,:,:3]
            img = img.astype(np.uint8)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) 
            
            mask_ = masks[coord_y[i] - (PSIZE[0]//2) : coord_y[i] + (PSIZE[0]//2), coord_x[i] - (PSIZE[0]//2) : coord_x[i] + (PSIZE[0]//2), np.newaxis]
            mask_ = mask_.astype(np.uint8) 
            if (0 in np.unique(img[:,:,0])) and (0 in np.unique(img[:,:,1])) and (0 in np.unique(img[:,:,2])):
                continue
            
            num = format(i, '05')
            ipath = Path(os.path.join(PATCH_SAVE, mode, name, str(level), f"{label}/img", f"{name}_{num}.png"))
            ipath.parent.mkdir(parents=True, exist_ok=True)
            
            try:
                cv2.imwrite(str(ipath), img)
            except:
                continue

In [47]:
a = np.arange(10)
print(a)
type(a)
np.random.shuffle(a)
print(a)
c = [4,2,3,5]
b= np.array([[0,1,0],[1,1,1],[0,1,0]])
print(b)
print(b[0][2])



[0 1 2 3 4 5 6 7 8 9]
[5 2 7 3 0 8 9 6 1 4]
[[0 1 0]
 [1 1 1]
 [0 1 0]]
0


In [46]:
b = 42
a = format(b, '05')
print(a)

00042


In [ ]:
# NOLN -> LN 순으로?? mrxs 파일과 xml파일들의 경로를 가져오는 것
for mode in modes:
    
    if mode == 'NOLN_metastasis':
        mrxs_fns = noln_mrxs_paths
        xml_fns = noln_xml_paths
    else:
        mrxs_fns = ln_mrxs_paths
        xml_fns = ln_xml_paths
        
    # zmrxs파일과 그 파일에 대응하는 xml파일을 tuple로 pair를 만듬    
    for (mrxs_fn, xml_fn) in tqdm(zip(mrxs_fns, xml_fns)): 
        mrxs_id = mrxs_fn.split('/')[-1].split('.')[0] ,#파일이름만 남도록
        xml_id = xml_fn.split('/')[-1].split('.')[0]
        name = mrxs_id #파일이름만 남은 것을 name변수에 저장
        assert mrxs_id == xml_id # 맞게 대응되어 묶였는지 확인
        assert os.path.isfile(mrxs_fn) == os.path.isfile(xml_fn) #경로도 맞게 묶였는지 확인
        slide = openslide.OpenSlide(mrxs_fn)
        for level in levels: #level별로 위에서 정의한 함수들을 이용하여 patch생성
            div = 2**level
            shape = load_mrxs_shape(slide, level = level)
            mask = get_normal(mrxs_fn, xml_fn, level, shape)
            make_patch(slide, mask, mode, name, level, "normal")
            mask = xml2mask(xml_fn, shape)
            make_patch(slide, mask, mode, name, level, "cancer")